In [1]:
import sys
import tensorflow as tf
import json
import os
n_epoch = 3
import pandas as pd

In [4]:
import pandas as pd
df = pd.read_csv('../RNN/tripadv.csv')
df.head(10)

,Ratings,Content
0,5.0,"Usually stay near the airport, but this trip w..."
1,4.0,Stayed at this Hilton for 2 nights. It was lik...
2,4.0,"Stayed there one night, December 16, on the wa..."
3,5.0,I just stayed here last weekend and have alrea...
4,5.0,My mother who is 90 and I stayed one night on ...
5,4.0,"A fabulous stop. Clean, friendly, excellent lo..."
6,4.0,I stopped here just for the night on a road tr...
7,4.0,Great spot close to shops and dinning but perh...
8,4.0,My husband and I go to Baltimore often as he h...
9,5.0,Super clean. Great hotel to stay at. We actual...


In [5]:
X = df.Content
y = df.Ratings
X.head(10)
y.head(10)

0    5.0
1    4.0
2    4.0
3    5.0
4    5.0
5    4.0
6    4.0
7    4.0
8    4.0
9    5.0
Name: Ratings, dtype: float64

In [6]:
def get_positive_or_negative_classes_two_classes(y):
    return y.apply(lambda val: 0 if val<=2 else 1)
def get_positive_or_negative_classes_three_classes(y):
    return y.apply(lambda val: 0 if val<=2 else 2 if val > 3 else 1)
def get_positive_or_negative_classes_five_classes(y):
    return y.apply(lambda val: 0 if val=='' else val)

In [7]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
y_train = get_positive_or_negative_classes_three_classes(y_train)
y_test = get_positive_or_negative_classes_three_classes(y_test)

In [8]:
import string
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1), token_pattern=r'\b\w{1,}\b')

In [9]:
vect.fit(X_train)
vocab = vect.vocabulary_
len(vocab)

132362

In [10]:
def convert_X_to_X_word_ids(X):
    return X.apply( lambda x: [vocab[w] for w in [w.lower().strip() for w in x.split()] if w in vocab] )

In [11]:
X_train_word_ids = convert_X_to_X_word_ids(X_train)
X_test_word_ids  = convert_X_to_X_word_ids(X_test)


In [16]:
X_train_word_ids.head()

87742     [61869, 117272, 96328, 91452, 103347, 53053, 6...
178022    [20262, 102137, 130008, 14055, 80615, 73925, 6...
214335    [57226, 4595, 118614, 126568, 117939, 127793, ...
115399    [128249, 112180, 117939, 61869, 43304, 118986,...
45982     [128249, 55082, 86553, 85218, 37384, 10077, 12...
Name: Content, dtype: object

In [17]:
X_train.head()

87742     Ancient, Budget-Minded Hotel The Ramada Plaza ...
178022    Beautiful rooms with american movie memorabili...
214335    Great 1st Time Visit This was my first visit t...
115399    We stayed this hotel due to a flight cancellat...
45982     We gave our olders daughter a trip to New York...
Name: Content, dtype: object

In [18]:
X_train_padded_seqs = pad_sequences(X_train_word_ids, maxlen=20, value=0)
X_test_padded_seqs  = pad_sequences(X_test_word_ids , maxlen=20, value=0)

In [69]:
print('X_train_padded_seqs.shape:', X_train_padded_seqs.shape)
print('X_test_padded_seqs.shape:', X_test_padded_seqs.shape)

('X_train_padded_seqs.shape:', (249481, 20))
('X_test_padded_seqs.shape:', (27721, 20))


In [11]:
pd.DataFrame(X_train_padded_seqs).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,117940,27635,14806,100844,16499,129388,126708,63449,61002,98215,44384,17548,117940,43140,83959,117940,52758,14806,17679,129388
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,51666,63449,14170,84176,10430,28576,62322,58600,17548,25872,63749,81749,50547,23913,62322,67114,110215,87187,64574,63449
3,63749,132436,65988,85956,56872,119676,126271,63449,131441,104520,119299,59703,89264,117940,21252,39522,85956,129653,132436,118618
4,63749,132436,82930,75287,132522,128890,119676,117940,129480,132436,79726,87187,85956,10430,27635,117940,72978,130345,82930,20370


In [12]:
pd.DataFrame(X_test_padded_seqs).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,63449,128501,74099,53247,119676,117940,57402,127950,12661,63449,58745,105785,67296,85956,117940,121126,63449,128501,84176,10684
1,81749,56404,117940,80712,85956,117897,14806,63449,23469,10430,112472,102671,17548,117940,16610,52890,87117,126271,17062,63449
2,118618,67114,10430,83179,95059,14806,10430,57663,73724,52890,126993,85396,117940,20249,14806,82292,117940,102755,16477,119519
3,90093,130345,14149,59851,63449,51666,112831,18753,84719,64574,106175,52890,8235,63449,58745,10430,131060,119299,25872,40435
4,121430,97478,96355,29180,48253,101862,64574,117940,60159,85396,78839,128107,67114,11661,117940,128176,41924,119676,102349,28296


In [19]:
unique_y_labels = list(y_train.value_counts().index)
unique_y_labels

[5.0, 4.0, 3.0, 2.0, 1.0]

In [41]:
y_train

array([[ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [13]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [14]:
def get_positive_or_negative_classes(y):
    return y.apply(lambda val: 0 if val<=2 else 2 if val > 3 else 0 )

In [15]:
# np_array = np.genfromtxt ('out.csv', delimiter=",")
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
max_words = 20
X_train_word_ids = convert_X_to_X_word_ids(X_train)
X_test_word_ids  = convert_X_to_X_word_ids(X_test)
X_train_word_ids = np.array(X_train_word_ids)
X_test_word_ids = np.array(X_test_word_ids)
y_train = get_positive_or_negative_classes_three_classes(y_train)
y_test = get_positive_or_negative_classes_three_classes(y_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train_padded_seqs = sequence.pad_sequences(X_train_word_ids, maxlen=max_words, value=0)
X_test_padded_seqs  = sequence.pad_sequences(X_test_word_ids , maxlen=max_words, value=0)

In [16]:
model = Sequential()
top_words = len(vocab)
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 32)            4235584   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               80250     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 4,319,189
Trainable params: 4,319,189
Non-trainable params: 0
_________________________________________________________________


In [17]:
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 32)            4235584   
_________________________________________________________________
flatten_2 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               160250    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 4,396,085
Trainable params: 4,396,085
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train_padded_seqs, y_train, validation_data=(X_test_padded_seqs, y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test_padded_seqs, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


Train on 249050 samples, validate on 27673 samples
Epoch 1/2
219s - loss: -8.3519e+00 - acc: 0.1517 - val_loss: -9.0058e+00 - val_acc: 0.1802
Epoch 2/2


In [49]:
import numpy as np
from sklearn import metrics
pred_classes = [round(i) for i in model.predict(X_test_padded_seqs)]
true_classes = y_test.tolist()
print('\nCNN Classifier\'s Accuracy: %0.5f\n' % metrics.accuracy_score(true_classes, pred_classes))
print ('Confusion Matrix:', metrics.confusion_matrix(true_classes, pred_classes))
print ('Classification Report')
print metrics.classification_report(true_classes, pred_classes)


CNN Classifier's Accuracy: 0.86821

('Confusion Matrix:', array([[ 1990,  2551],
       [ 1096, 22036]]))
Classification Report
             precision    recall  f1-score   support

          0       0.64      0.44      0.52      4541
          1       0.90      0.95      0.92     23132

avg / total       0.85      0.87      0.86     27673

